In [1]:
% config Completer.use_jedi = False

UsageError: Line magic function `%` not found.


## Import the Dependencies

In [7]:
import cv2
import numpy as np
import keras
import math

from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf

from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,
    Dropout, Dense, Input, concatenate,
    GlobalAveragePooling2D, AveragePooling2D,
    Flatten
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

IndentationError: unexpected indent (<ipython-input-7-472d4776d380>, line 13)

## Create the Inception module

In [3]:
def inception_module(x, kernel_init, bias_init,
                     filters_x1,
                     filters_x3_reduce, filters_x3,
                     filters_x5_reduce, filters_x5,
                     filters_pool_proj,
                     name=None):
    conv_x1 = Conv2D(filters=filters_x1, kernel_size=(1, 1), padding='same', activation='relu',
                     kernel_initializer=kernel_init, bias_initializer=bias_init)(x)

    conv_x3_red = Conv2D(filters=filters_x3_reduce, kernel_size=(1, 1), padding='same', activation='relu',
                         kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_x3 = Conv2D(filters=filters_x3, kernel_size=(3, 3), padding='same', activation='relu',
                     kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_x3_red)

    conv_x5_red = Conv2D(filters=filters_x5_reduce, kernel_size=(1, 1), padding='same', activation='relu',
                         kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_x5 = Conv2D(filters=filters_x5, kernel_size=(5, 5), padding='same', activation='relu',
                     kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_x5_red)

    pool_proj_red = MaxPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters=filters_pool_proj, kernel_size=(1, 1), padding='same', activation='relu',
                       kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj_red)

    return concatenate([conv_x1, conv_x3, conv_x5, pool_proj], axis=3, name=name)

## Prepare the data

In [4]:
num_classes = 10


def load_cifar10_data(img_rows, img_cols):
    # load cifar10 training and validation sets
    (x_train, y_train), (x_valid, y_valid) = cifar10.load_data()

    # resize training images
    x_train = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_train])
    x_valid = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_valid])

    # transform targets to keras compatible format
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_valid = np_utils.to_categorical(y_valid, num_classes)

    # preprocess data
    x_train = x_train / 255.0
    y_valid = x_valid / 255.0

    return x_train, y_train, x_valid, y_valid

In [ ]:
x_train, y_train, x_test, y_test = load_cifar10_data(224, 224)

## Create the model

In [5]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

NameError: name 'keras' is not defined

In [8]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(filters=64, kernel_size=(7, 7), padding='same',
           strides=(2, 2), activation='relu', name='conv_1_7x7/2',
           kernel_initializer=kernel_init, bias_initializer=bias_init)
x = MaxPool2D(pool_size=(3, 3), padding='same', strides=(2, 2),
              name='max_pool_1_3x3/2')(x)
x = Conv2D(filters=192, kernel_size=(3, 3), padding='same',
           strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_x1=64,
                     filters_x3=128, filters_x3_reduce=96,
                     filters_x5=32, filters_x5_reduce=16,
                     filters_pool_proj=32,
                     name='inception_3a')
x = inception_module(x,
                     filters_x1=128,
                     filters_x3=192, filters_x3_reduce=128,
                     filters_x5=96, filters_x5_reduce=32,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D(pool_size=(3, 3), padding='same',
              strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_x1=192,
                     filters_x3=208, filters_x3_reduce=96,
                     filters_x5=48, filters_x5_reduce=16,
                     filters_pool_proj=64,
                     name='inception_4')

classifier_1 = AveragePooling2D((5, 5), strides=3)(x)
classifier_1 = Conv2D(filters=128, kernel_size=(1, 1), padding='same',
                      activation='relu')(classifier_1)
classifier_1 = Flatten()(classifier_1)
classifier_1 = Dropout(0.7)(classifier_1)
classifier_1 = Dense(1024, activation='relu')(classifier_1)
classifier_1 = Dropout(0.7)(classifier_1)
classifier_1 = Dense(10, activation='softmax', 
                     name='auxilliary_output_1')(classifier_1)



NameError: name 'Input' is not defined